## Install Packages

In [6]:
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import pitching_stats
from pybaseball import batting_stats
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np
import pyodbc
from pybaseball import cache
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import psycopg2
from sqlalchemy import create_engine
import pyodbc
from dateutil.relativedelta import relativedelta
from datetime import date

In [2]:
param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}

### Connect to Database

In [3]:
param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}

def connect(params_dict):
    "Connecting to PostgreSQL Database Server"
    conn = None
    try:
        print("Connecting to the PostgreSQL Database")
        conn = psycopg2.connect(**params_dict)
    except (Exception, psycopg2.DataError) as error:
        print(error)
    return conn

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dict['user'],
    param_dict['password'],
    param_dict['host'],
    param_dict['database']
)
engine = create_engine(connect)

### Create Functions for getting and appending data

In [4]:
def get_data(start_date, end_date):
    cache.enable()
    df = statcast(start_dt = start_date, end_dt = end_date)
    df = df[['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name',  'batter', 'pitcher', 'events', 'description', 'spin_dir', 
        'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team',
         'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning',
         'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 
         'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5',  
         'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle',  'estimated_woba_using_speedangle', 'woba_value', 
         'woba_denom', 'babip_value', 'iso_value', 'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name', 'home_score', 'away_score', 'bat_score', 'fld_score', 
          'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis', 'delta_home_win_exp', 'delta_run_exp', 
         'bat_speed', 'swing_length', 'estimated_slg_using_speedangle',  'delta_pitcher_run_exp', 'hyper_speed', 'home_score_diff',  'bat_score_diff', 'home_win_exp', 'bat_score_diff', 'home_win_exp', 
         'bat_win_exp', 'age_pit_legacy', 'age_bat_legacy', 'age_pit', 'age_bat', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat', 'pitcher_days_since_prev_game',
         'batter_days_since_prev_game', 'batter_days_until_next_game', 'api_break_z_with_gravity', 'api_break_x_arm', 'api_break_x_batter_in', 'arm_angle']]
    return df
    
def append_data(data):
    data.to_sql(
        name = 'raw_pitch', #table name
        con = engine,
        if_exists= 'append',
        index = False
    )
    return print("Appended Data")

def get_dates(year, month, day):
    date_list = []
    n = 14
    for i in range(0,n):
        temp_date = date(year,month,day) + relativedelta(years=i)
        date_list.append(str(temp_date))
    return date_list

#### Appending Data To tables

In [7]:
start_date = get_dates(2012,1,1)
end_date = get_dates(2012,12,31)
def tryAgain(retries=0):
    for i in range(0,len(start_date)):
        if retries > 15: return print("Too Many Attempts")
        try:
            print("Starting ", start_date[i])
            df = get_data(start_date[i], end_date[i])
            append_data(df)
            print("Data Appended for ", start_date[i], "to ", end_date[i])
            start_date.remove(start_date[i])
            end_date.remove(end_date[i])
        except:
            print("Failure for ", start_date[i], ".Number: ", retries)
            tryAgain(retries+1)
        #return print("Successfully appended all data")
tryAgain()

Starting  2012-01-01
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 215/215 [06:32<00:00,  1.83s/it]


Appended Data
Data Appended for  2013-01-01 to  2013-12-31
Starting  2014-01-01
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 222/222 [06:47<00:00,  1.83s/it]


Appended Data
Data Appended for  2015-01-01 to  2015-12-31
Starting  2016-01-01
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


...

100%|██████████| 246/246 [07:17<00:00,  1.78s/it]


Appended Data
Data Appended for  2023-01-01 to  2023-12-31
Starting  2024-01-01
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 246/246 [07:17<00:00,  1.78s/it]


Appended Data
Data Appended for  2025-01-01 to  2025-12-31


IndexError: list index out of range

#### Append Raw Player Lookup Data 

In [ ]:
batter_ids = pd.read_sql_query("select distinct batter  from raw_pitch", con=engine)
pitcher_ids = pd.read_sql_query("select distinct pitcher  from raw_pitch", con=engine)


In [35]:
batter_ids = pd.read_sql_query("select distinct batter  from raw_pitch", con=engine)
pitcher_ids = pd.read_sql_query("select distinct pitcher  from raw_pitch", con=engine)
batters = batter_ids['batter'].unique()
pitchers = pitcher_ids['pitcher'].unique()
batter_df = playerid_reverse_lookup(batters, key_type='mlbam')
pitcher_df = playerid_reverse_lookup(pitchers, key_type='mlbam')
batter_df.to_sql(name = 'raw_player_lookup', #table name
        con = engine,
        if_exists= 'append',
        index = False)
pitcher_df.to_sql(name = 'raw_player_lookup', #table name
        con = engine,
        if_exists= 'append',
        index = False)
print("All Data Appended")

All Data Appended


In [18]:
def tryAgain(retries=0):
    for i in range(0,len(start_date)):
        if retries > 15: return print("Too Many Attempts")
        try:
            print("Starting ", start_date[i])
            df = get_data(start_date[i], end_date[i])
            append_data(df)
            print("Data Appended for ", start_date[i], "to ", end_date[i])
            start_date.remove(start_date[i])
            end_date.remove(end_date[i])
        except:
            print("Failure for ", start_date[i], ".Number: ", retries)
            tryAgain(retries+1)
        #return print("Successfully appended all data")
tryAgain()